<a href="https://colab.research.google.com/github/hi13i/DLNoteClass/blob/master/DLNoteClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 最初にやるやつ

In [ ]:
!git clone https://github.com/hi13i/DLNoteClass.git


Cloning into 'DLNoteClass'...
fatal: could not read Username for 'https://github.com': No such device or address


In [5]:
!pip3 install --pre torch torchvision --extra-index-url https://download.pytorch.org/whl/nightly/cpu
!pip3 install torchinfo
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchinfo import summary
from sklearn.metrics import classification_report
import math
import time
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import os

import cv2


torch.manual_seed(17)

if not torch.cuda.is_available():
  print('\nGPU が割り当てられていません.')
  print('「編集」>「ノートブックの設定」から「ハードウェア アクセラレータ」を GPU に変更して再実行してください.')
  print('なお「GPU バックエンドに接続できません」と表示される場合にはしばらく経過してから再実行してください.')
else:
  print('\nGPU が利用できます.')
  print('無料版では GPU として Tesla K80 か Tesla T4 が割り当てられます.')
  print('Tesla T4 の方が高性能で学習も速いですが、何度も利用しているユーザには Tesla K80 が割り当てられることが多いようです.\n')
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  print(gpu_info)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/nightly/cpu
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

GPU が利用できます.
無料版では GPU として Tesla K80 か Tesla T4 が割り当てられます.
Tesla T4 の方が高性能で学習も速いですが、何度も利用しているユーザには Tesla K80 が割り当てられることが多いようです.

Tue Jun 13 05:39:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 

In [3]:
!unzip gray.zip

Archive:  gray.zip
replace gray/Eight/gray_e1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# つぎに

In [7]:
test = 'gray/Eight/gray_e1.jpg'
# 対象画像読み込み
img = cv2.imread("gray/Eight/gray_e1.jpg",cv2.IMREAD_COLOR)

# 画像の大きさを取得
height, width, channels = img.shape[:3]
print("width: " + str(width))
print("height: " + str(height))

width: 64
height: 64


In [10]:
data_dir = "gray"
import torch
import torchvision
from torchvision import transforms

# image_sizeやmean, stdはデータに合わせて設定してください。
image_size = 224
mean = (0.5)
std = (0.5)

# trainデータとvalidationデータが入っているディレクトリのパスを指定
train_image_dir = 'gray'
val_image_dir = 'gray2'

# trainデータ向けとvalidationデータ向けに、transformを用意します。
# 皆さんのやりたいことに合わせて適宜変更してください。
data_transform = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(
            image_size, scale=(0.5, 1.0)
        ),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=[-15, 15]),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
        transforms.RandomErasing(0.5),
    ]),
    'val': transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
}

# torchvision.datasets.ImageFolderでデータの入っているディレクトリのパスと
# transformを指定してあげるだけ。
train_dataset = torchvision.datasets.ImageFolder(root=train_image_dir, transform=data_transform['train'])
val_dataset = torchvision.datasets.ImageFolder(root=val_image_dir, transform=data_transform['val'])

# Datasetができたら、dataloaderに渡してあげればOK
batch_size = 32
train_dataLoader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)
val_dataLoader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False
)

FileNotFoundError: ignored